<a href="https://colab.research.google.com/github/AhmadIsmail83N/MyData/blob/master/UC_Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ensure using TensorFlow 1.x**

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


**Import Libraries**

In [0]:
import numpy as np
import tensorflow as tf
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

**Get Datasets according to variant 14**

In [3]:
categories = ["comp.sys.ibm.pc.hardware","rec.motorcycles","sci.electronics","alt.atheism"]
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
print('total texts in train:',len(newsgroups_train.data))
print('total texts in test:',len(newsgroups_test.data))
print('text',newsgroups_train.data[0])
print('category:',newsgroups_train.target[0])

total texts in train: 2259
total texts in test: 1502
text From: bil@okcforum.osrhe.edu (Bill Conner)
Subject: Re: Americans and Evolution
Nntp-Posting-Host: okcforum.osrhe.edu
Organization: Okcforum Unix Users Group
X-Newsreader: TIN [version 1.1 PL9]
Lines: 30

Robert Singleton (bobs@thnext.mit.edu) wrote:

: > Sure it isn't mutually exclusive, but it lends weight to (i.e. increases
: > notional running estimates of the posterior probability of) the 
: > atheist's pitch in the partition, and thus necessarily reduces the same 
: > quantity in the theist's pitch. This is because the `divine component' 
: > falls prey to Ockham's Razor, the phenomenon being satisfactorily 
:                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
: > explained without it, and there being no independent evidence of any 
:   ^^^^^^^^^^^^^^^^^^^^
: > such component. More detail in the next post.
: > 

Occam's Razor is not a law of nature, it is way of analyzing an
argument, even so, it interesti

**Generate Array of words from text of each collection**

In [0]:
vocab = Counter()
for text in newsgroups_train.data:
  for word in text.split(' '):
      vocab[word.lower()] += 1

for text in newsgroups_test.data:
  for word in text.split(' '):
      vocab[word.lower()] += 1

total_words = len(vocab)

def get_word_2_index(vocab):
  word2index = {}
  for i, word in enumerate(vocab):
    word2index[word.lower()] = i
  return word2index

word2index = get_word_2_index(vocab)

def get_batch(df, i, batch_size):
  batches = []
  results = []
  texts = df.data[i * batch_size:i * batch_size + batch_size]
  categories = df.target[i * batch_size:i * batch_size + batch_size]
  for text in texts:
    layer = np.zeros(total_words, dtype=float)
    for word in text.split(' '):
      layer[word2index[word.lower()]] += 1
    batches.append(layer)

  for category in categories:
    y = np.zeros((3), dtype=float)
    if category == 0:
      y[0] = 1.
    elif category == 1:
      y[1] = 1.
    else:
      y[2] = 1.
    results.append(y)

  return np.array(batches), np.array(results)

**Neural NetWork: Parameters and design : 3 hidden layers**

In [0]:
# Параметры обучения
learning_rate = 0.07
training_epochs = 10
batch_size = 150
display_step = 1
# Network Parameters
n_hidden_1 = 35 # скрытый слой
n_hidden_2 = 25 # скрытый слой
n_hidden_3 = 20 # скрытый слой
n_input = total_words # количество уникальных слов в наших текстах
n_classes = 3 # 3 класса
input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output")
def multilayer_perceptron(input_tensor, weights, biases):
  # скрытый слой
  layer_1_multiplication = tf.matmul(input_tensor, weights['h1'])
  layer_1_addition = tf.add(layer_1_multiplication, biases['b1'])
  layer_1 = tf.nn.relu(layer_1_addition)
  # скрытый слой
  layer_2_multiplication = tf.matmul(layer_1, weights['h2'])
  layer_2_addition = tf.add(layer_2_multiplication, biases['b2'])
  layer_2 = tf.nn.relu(layer_2_addition)
  # скрытый слой
  layer_3_multiplication = tf.matmul(layer_2, weights['h3'])
  layer_3_addition = tf.add(layer_3_multiplication, biases['b3'])
  layer_3 = tf.nn.relu(layer_3_addition)
  # выходной слой
  out_layer_multiplication = tf.matmul(layer_3, weights['out'])
  out_layer_addition = out_layer_multiplication + biases['out']
  return out_layer_addition

In [0]:
# инициализация параметров сети
weights = {
'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
'b1': tf.Variable(tf.random_normal([n_hidden_1])),
'b2': tf.Variable(tf.random_normal([n_hidden_2])),
'b3': tf.Variable(tf.random_normal([n_hidden_3])),
'out': tf.Variable(tf.random_normal([n_classes]))
}
# создание модели
prediction = multilayer_perceptron(input_tensor, weights, biases)
# Фукнция потерь
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=output_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
init = tf.global_variables_initializer()

In [19]:
# Запуск
with tf.Session() as sess:
  sess.run(init)
  # Цикл обучения
  for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Проход по всем батчам
    for i in range(total_batch):
      batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
      c,_ = sess.run([loss,optimizer], feed_dict={input_tensor: batch_x,output_tensor:batch_y})
      # Вычисляем среднее фукнции потерь
      avg_cost += c / total_batch
    print("Эпоха:", '%04d' % (epoch+1), "loss=", "{:.16f}".format(avg_cost))
  print("Обучение завершено!")

  # Тестирование
  correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
  # Расчет точности
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  total_test_data = len(newsgroups_test.target)
  batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
  print("Точность:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))

Эпоха: 0001 loss= 370.3777043660481354
Эпоха: 0002 loss= 37.0278567314147864
Эпоха: 0003 loss= 10.0832759857177727
Эпоха: 0004 loss= 9.8718433698018391
Эпоха: 0005 loss= 18.3314972122510227
Эпоха: 0006 loss= 7.9368372639020279
Эпоха: 0007 loss= 7.2169995983441666
Эпоха: 0008 loss= 3.8342475454012548
Эпоха: 0009 loss= 1.0332314416766166
Эпоха: 0010 loss= 0.4355002654095490
Обучение завершено!
Точность: 0.75233024
